In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from segmentation_models import Unet

import numpy as np
import matplotlib.pyplot as plt



### Define separate datagenerators for masks and images

In [9]:
options = {'shear_range': 5.,
           'zoom_range': 0.5,
           'horizontal_flip': True,
           'vertical_flip': True
          }
image_datagen = ImageDataGenerator(
    rescale=1./255,
    **options
)
mask_datagen = ImageDataGenerator(
    **options
)

### Flow from directory

use the same seed for mask and images in order to give them the same transformations.

I think that as it currently is implemented this unet expects the mask to be shape (224, 224, N_classes) and the last dimension is basically a one-hot encoding. We probably need a preprocessing function on the mask data to implement this properly. Alternatively follow the tensorflow example https://www.tensorflow.org/tutorials/images/segmentation. where the shape would be (224,224,1) and the last dimension is an integer, we would have to modify the loss function.

Though here we need to choose an approriate loss function

Also, currently the zip works as a generator but it doesn't give  length so training just says: `2/unknown`. It should be pretty easy to make our own small generator class. I think it just needs to implement `__yield__` and `__len__` where the len will return a batch size.

TODO:

1. Process masks in accordance with the model's expectations
2. Figure out correct loss function
3. (nice to have) generator class

In [10]:
# original image is 450x540
# these dimensions need to be divisible by 32
target_x = 224
target_y = 224
target_size = (target_x, target_y)

seed = np.random.randint(0,1e5)
gen_x = image_datagen.flow_from_directory('imgs',seed=seed, target_size=target_size,class_mode=None)
gen_y = mask_datagen.flow_from_directory('masks',seed=seed, target_size=target_size,class_mode=None)#,color_mode='grayscale')

Found 13 images belonging to 1 classes.
Found 13 images belonging to 1 classes.


In [11]:
tf.keras.backend.set_image_data_format('channels_last')
model = Unet('resnet18', encoder_weights='imagenet',
                 classes=1,
                 input_shape=(target_x, target_y, 3))

model.compile('Adam', 'categorical_crossentropy', ['binary_accuracy'])

In [ ]:


model.fit(zip(gen_x, gen_y))